In [1]:
import pandas as pd
import numpy as np
import requests as re
import bs4

# 2023

In [23]:
teams = ['grand-canyon', 'michigan', 'tennessee', 'arizona', 'unlv', 'north-dakota-state', 'washington-state', 'north-dakota-state',
        'santa-clara', 'santa-clara', 'santa-clara', 'northern-colorado', 'northern-colorado', 'northern-colorado', 'uc-riverside',
        'long-beach-state', 'long-beach-state', 'long-beach-state', 'csu-bakersfield', 'csu-bakersfield', 'csu-bakersfield',
        'san-diego-state', 'uc-irvine', 'uc-irvine', 'uc-irvine', 'usd', 'uc-davis', 'uc-davis', 'uc-davis', 'san-diego-state',
        'san-diego-state', 'hawaii', 'hawaii', 'hawaii', 'uc-santa-barbara', 'uc-santa-barbara', 'uc-santa-barbara', 'usc',
        'cal-poly', 'cal-poly', 'cal-poly', 'byu', 'csun', 'csun', 'csun', 'cal-state-fullerton', 'cal-state-fullerton',
        'cal-state-fullerton', 'san-diego-state', 'uc-riverside', 'uc-riverside', 'uc-riverside']
nums = list(range(36, 42)) + list(range(43, 89))
triton_innings23 = []
for i in range(52):
    play_by_play_link = 'https://ucsdtritons.com/sports/baseball/stats/2023/' + teams[i] + '/boxscore/93' + str(nums[i])
    play_soup = bs4.BeautifulSoup(re.get(play_by_play_link).content)
    play_by_play_html = play_soup.find('section', attrs={'id': 'play-by-play'})
    innings = play_by_play_html.find('div', attrs={'id': 'inning-all'}).find_all('div', attrs={'class': 'panel relative'})
    triton_innings23 += (list(filter(lambda x: x.find('caption').text.find('UC San Diego') > -1, innings)))
triton_innings23

[<div class="panel relative">
 <div class="row pad">
 <table class="sidearm-table play-by-play">
 <caption>UC San Diego - Bottom of 1st</caption>
 <thead>
 <tr>
 <th class="text-hide" scope="col">Play Description</th>
 <th class="text-center" scope="col">GCU</th>
 <th class="text-center" scope="col">UCS</th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td>Saunders, L. flied out to lf (1-1 BK).</td>
 <td class="text-center">2</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Smith, J. struck out looking (1-2 BKFFFK).</td>
 <td class="text-center">2</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Halbach, M. struck out swinging (0-2 KFS).</td>
 <td class="text-center">2</td>
 <td class="text-center">0</td>
 </tr>
 </tbody>
 </table>
 <dl class="special-stats emphasize inline text-right">
 <dt>Runs</dt>
 <dd>0</dd>
 <dt>Hits</dt>
 <dd>0</dd>
 <dt>Errors</dt>
 <dd>0</dd>
 <dt>Left On Base</dt>
 <dd>0</dd>
 </dl>
 </div>
 </div>,
 <div class="panel relative">
 <div class="row pad">
 <table

Data to be extracted for each plate appearance:

- Name of hitter
- Number of outs
- Runners (keep track in 3 columns - either boolean or binary for each base)
- Score
- Tritons Away/Home
- Total Number of runs scored in inning
- Number of runs scored up to that plate appearance in inning

To be added later

- At least 1 run scored in inning (bool)
- wOBA of hitter
- Number of runs scored after plate appearance (including hitter currently up, so if there was 1 run already scored, and the current hitter hits a 2 run homer, the 2 runs count towards this data point)

Order of columns: batter, first, second, third, outs, away_score, home_score, Triton_side, total_runs_inn, runs_up_to_batter

Notes
- Parentheses in line means an at bat occurred and ended - start with this to indicate which block of code the line should be run through - some play by plays don't have the parentheses
- Line always starts with batter name in format Last, First initial.
- it will say if a runner scores, use the word 'scored' as indicator of how many runs were scored, not RBI (ex: Smith, J. singled through left side, RBI (2-1 BBK); Glascoe, L. advanced to second; Saunders, L. advanced to third; Kleszcz, B. scored.)
- semicolons indicate more happened on play, period at end of line indicates everything that happened has been stated

Outcomes
- All stated in the past tense (struck out, flied out, stole second, advanced to third, etc)
- strikeout
- flyout
- pop out (weird example: Sudyka, N. popped up to rf)
- lineout
- ground out
- foul out
- double play
- fielder's choice
- reach on error
- single
- double
- triple
- homerun
- walk
- intentional walk
- hit by pitch
- catcher's interference
- dropped 3rd strike

Non-at bat outcomes
- wild pitch
- stolen base
- fielder's indifference
- passed ball
- picked off
- caught stealing

In [3]:
def isbatter(txt):
    first_event = txt.split(';')[0]
    #if (first_event.find('advanced') > -1) and (first_event.find('on the throw') > -1):
        #return True
    if (first_event.find('stole') > -1) or ((first_event.find('advanced') > -1) and (first_event.split(' ')[2] == 'advanced')) or (first_event.find('scored') > -1) or (first_event.find('caught stealing') > -1) or (first_event.find('picked off') > -1):
        return False
    else:
        return True

In [4]:
batter_plays = []
non_batter_plays = []
for inns in triton_innings:
    pbp = inns.find_all('td')
    pbp_tags = [i for i in pbp if not i.attrs]
    pbp_text = [i.text for i in pbp_tags]
    for play in pbp_text:
        if isbatter(play):
            batter_plays.append(play)
        else:
            non_batter_plays.append(play)

In [44]:
names23 = ['Hackworth', 'Fuhrman', 'Halbach', 'Kane', 'Gonzalez', 'Larson', 'Sudyka', 'Kleszcz', 'Saunders', 'Smith', 'Costello',
        'Ujimori', 'Adame', 'Solis', 'Robinson', 'Preap', 'Lomanto', 'Glascoe']
def get_situations(triton_innings, names):
    pbp_data = []
    for inn in triton_innings:
        first = '_'
        second = '_'
        third = '_'
        outs = 0
        tot_runs = int(inn.find('dd').text)
        runs_so_far = 0
        #if inn.find('caption').text.find('Top') > -1:
            #home_away = 'Away'
        #else:
            #home_away = 'Home'
        #score = inn.find_all('td', attrs={'class': 'text-center'})[:2]
        #away_score = int(score[0].text)
        #home_score = int(score[1].text)
        pbp = inn.find_all('td')
        pbp_tags = [i for i in pbp if ((not i.attrs) or (i.attrs['class'][0] == 'text-italic'))]
        pbp_text = [i.text for i in pbp_tags if ((not i.attrs) or (i.attrs['class'][0] == 'text-italic' and i.text.find('pinch ran') > -1))]
        for play in pbp_text:
            if play.split()[0].strip(',') not in names:
                continue
            if play.find('pinch ran') > -1:
                new_runner = play.split(' ')[0][:-1]
                if new_runner == 'Smith':
                    new_runner = play.split(' ')[1].strip('.') + ' ' + new_runner
                old_runner = play.split(' ')[-2][:-1]
                if old_runner == 'Smith':
                    old_runner = play.split(' ')[-1].strip('.') + ' ' + old_runner
                if first == old_runner:
                    first = new_runner
                elif second == old_runner:
                    second = new_runner
                else:
                    third = new_runner
            elif isbatter(play):
                batter = play.split()[0].strip(',')
                if batter == 'Smith':
                    batter = play.split(' ')[1].strip('.') + ' ' + batter
                situation = [batter, first, second, third, outs, tot_runs, runs_so_far]
                pbp_data.append(situation)
                for event in play.split(';'):
                    player = event.split()[0].strip(',')
                    if player == 'Smith':
                        if event.find('A. ') > -1:
                            player = 'A Smith'
                        else:
                            player = 'J Smith'
                    if event.find('reached') > -1:
                        if event.find('advanced') == -1:
                            first = player
                        elif event.find('second') > -1:
                            second = player
                        else:
                            third = player
                    elif (event.find('double play') > -1) or (event.find('out') > -1) or (event.find('popped up') > -1):
                        outs += 1
                        if first == player:
                            first = '_'
                        if second == player:
                            second == '_'
                        if third == player:
                            third == '_'
                    elif event.find('advanced') > -1:
                        if (event.find('second') > -1) and (event.find('third') == -1):
                            second = player
                            if first == player:
                                first = '_'
                            if third == player:
                                third = '_'
                        else:
                            third = player
                            if first == player:
                                first = '_'
                            if second == player:
                                second = '_'
                    elif (event.find('singled') > -1) or (event.find('walked') > -1) or (event.find('hit by pitch') > -1):
                        first = player
                    elif event.find('doubled') > -1:
                        second = player
                    elif event.find('tripled') > -1:
                        third = player
                    elif event.find('homered') > -1:
                        first = '_'
                        second = '_'
                        third = '_'
                        runs_so_far += 1
                        #if home_away == 'Away':
                            #away_score += 1
                        #else:
                            #home_score += 1
                    elif event.find('scored') > -1:
                        runs_so_far += 1
                        #if home_away == 'Away':
                            #away_score += 1
                        #else:
                            #home_score += 1
                        if first == player:
                            first = '_'
                        if second == player:
                            second = '_'
                        if third == player:
                            third = '_'
                    else:
                        continue
            else:
                for event in play.split(';'):
                    player = event.split()[0].strip(',')
                    if player == 'Smith':
                        if event.find('A. ') > -1:
                            player = 'A Smith'
                        else:
                            player = 'J Smith'
                    if event.find('out') > -1:
                        outs += 1
                        if first == player:
                            first = '_'
                        if second == player:
                            second = '_'
                        if third == player:
                            third = '_'
                    elif (event.find('advanced') > -1) or (event.find('stole') > -1):
                        if first == player:
                            first = '_'
                        if second == player:
                            second = '_'
                        if third == player:
                            third = '_'
                        if (event.find('second') > -1) and (event.find('third') == -1):
                            second = player
                        elif event.find('third') > -1:
                            third = player
                        else:
                            runs_so_far += 1
                            #if home_away == 'Away':
                                #away_score += 1
                            #else:
                                #home_score += 1
                    elif event.find('scored') > -1:
                        runs_so_far += 1
                        #if home_away == 'Away':
                            #away_score += 1
                        #else:
                            #home_score += 1
                        if first == player:
                            first = '_'
                        if second == player:
                            second = '_'
                        if third == player:
                            third = '_'
                    else:
                        continue

    situations = pd.DataFrame(data = pbp_data, columns = ['batter', 'first', 'second', 'third', 'outs', 'total_runs_inn', 'runs_up_to_batter'])
    #'away_score', 'home_score', 'triton_side',
    # away_score, home_score, home_away, 
    return situations

situations23 = get_situations(triton_innings23, names23)
situations23

,batter,first,second,third,outs,total_runs_inn,runs_up_to_batter
0,Saunders,_,_,_,0,0,0
1,J Smith,_,_,_,1,0,0
2,Halbach,_,_,_,2,0,0
3,Kane,_,_,_,0,0,0
4,Fuhrman,Kane,_,_,0,0,0
...,...,...,...,...,...,...,...
2150,Costello,_,Sudyka,_,1,1,1
2151,Halbach,_,Sudyka,_,2,1,1
2152,Kane,_,_,_,0,0,0
2153,Fuhrman,_,_,_,1,0,0


In [45]:
situations23['runs_after_batter'] = situations23['total_runs_inn'] - situations23['runs_up_to_batter']
situations23['at_least_1_scored'] = situations23['runs_after_batter'] >= 1
#wOBA of hitter to be added
situations23.iloc[24:36]

,batter,first,second,third,outs,total_runs_inn,runs_up_to_batter,runs_after_batter,at_least_1_scored
24,Sudyka,Gonzalez,Fuhrman,Halbach,2,0,0,0,False
25,Kleszcz,_,_,_,0,4,0,4,True
26,Glascoe,Kleszcz,_,_,0,4,0,4,True
27,Saunders,Glascoe,Kleszcz,_,0,4,0,4,True
28,J Smith,Saunders,Glascoe,Kleszcz,0,4,0,4,True
29,Halbach,J Smith,Saunders,Glascoe,0,4,1,3,True
30,Kane,Halbach,J Smith,Saunders,0,4,2,2,True
31,Fuhrman,Kane,Halbach,_,0,4,4,0,False
32,Gonzalez,Kane,Halbach,_,1,4,4,0,False
33,Sudyka,Kane,_,Halbach,2,4,4,0,False


In [7]:
situations_base = situations23.copy()
situations_base['first'] = situations_base['first'].apply(lambda x: '1' if x != '_' else x)
situations_base['second'] = situations_base['second'].apply(lambda x: '2' if x != '_' else x)
situations_base['third'] = situations_base['third'].apply(lambda x: '3' if x != '_'else x)
situations_base['base'] = situations_base['first'] + ' ' + situations_base['second'] + ' ' + situations_base['third']
situations_base.to_csv(path_or_buf = 'baseball_atbats_2023.csv', index = False)
situations_base[24:36]

,batter,first,second,third,outs,total_runs_inn,runs_up_to_batter,runs_after_batter,at_least_1_scored,base
24,Sudyka,1,2,3,2,0,0,0,False,1 2 3
25,Kleszcz,_,_,_,0,4,0,4,True,_ _ _
26,Glascoe,1,_,_,0,4,0,4,True,1 _ _
27,Saunders,1,2,_,0,4,0,4,True,1 2 _
28,Smith,1,2,3,0,4,0,4,True,1 2 3
29,Halbach,1,2,3,0,4,1,3,True,1 2 3
30,Kane,1,2,3,0,4,2,2,True,1 2 3
31,Fuhrman,1,2,_,0,4,4,0,False,1 2 _
32,Gonzalez,1,2,_,1,4,4,0,False,1 2 _
33,Sudyka,1,_,3,2,4,4,0,False,1 _ 3


In [8]:
# create run expectancy matrix and run probability matrix using pivot tables

In [9]:
re_matrix = pd.pivot_table(data = situations_base, values = ['runs_after_batter'], index = ['base'], columns = ['outs'], aggfunc = 'mean')
re_matrix

runs_after_batter                    
outs                  0         1         2
base                                       
1 2 3          3.315789  2.166667  0.685714
1 2 _          1.744186  1.000000  0.640449
1 _ 3          2.705882  2.151515  0.655172
1 _ _          1.177778  0.697368  0.253623
_ 2 3          2.833333  2.187500  0.600000
_ 2 _          1.341463  0.686567  0.369565
_ _ 3          1.400000  1.400000  0.536585
_ _ _          0.746269  0.404040  0.110092

In [10]:
rp_matrix = pd.pivot_table(data = situations_base, values = ['at_least_1_scored'], index = ['base'], columns = ['outs'], aggfunc = 'mean')
rp_matrix

at_least_1_scored                    
outs                  0         1         2
base                                       
1 2 3          0.842105  0.777778  0.257143
1 2 _          0.744186  0.400000  0.247191
1 _ 3          1.000000  0.787879  0.310345
1 _ _          0.496296  0.309211  0.108696
_ 2 3          0.916667  0.781250  0.250000
_ 2 _          0.609756  0.432836  0.217391
_ _ 3          0.900000  0.733333  0.365854
_ _ _          0.339019  0.195286  0.059633

Risk x runs to gain y runs - find odds to make it work the risk like poker

Example:
Risk 0.33 runs to gain 0.66 runs - 2/1 ratio means if there are >50% odds for success, the play produces a positive expected value

Another option (choices with more than 2 outcomes) - total expected risk/reward based on percentage of each outcome occurring

From [article](https://www.pitcherlist.com/adjusting-for-the-current-run-expectancy-matrix/): break even point = |failure - current| / (|failure - current| + |success - current|)

In [11]:
def calc_be_point(current, success, failure = 0, matrix = 're'):
    if matrix == 're':
        s_runs = re_matrix.loc[success['base']].loc['runs_after_batter'].loc[success['outs']]
        c_runs = re_matrix.loc[current['base']].loc['runs_after_batter'].loc[current['outs']]
        if failure != 0:
            f_runs = re_matrix.loc[failure['base']].loc['runs_after_batter'].loc[failure['outs']]
        else:
            f_runs = 0
    else:
        s_runs = rp_matrix.loc[success['base']].loc['at_least_1_scored'].loc[success['outs']]
        c_runs = rp_matrix.loc[current['base']].loc['at_least_1_scored'].loc[current['outs']]
        if failure != 0:
            f_runs = rp_matrix.loc[failure['base']].loc['at_least_1_scored'].loc[failure['outs']]
        else:
            f_runs = 0
    risk = c_runs - f_runs
    reward = s_runs - c_runs
    if reward < 0:
        return 'negative expected value play'
    break_even = risk / (risk + reward)
    return break_even
    
calc_be_point({'base': '1 _ _', 'outs': 0}, {'base': '_ 2 _', 'outs': 1}, {'base': '1 _ _', 'outs': 1}, matrix = 're')

'negative expected value play'

In [26]:
import plotly.graph_objects as go

# Define your single value (between 0 and 100)
value = 75

# Create an indicator trace
trace = go.Indicator(
    mode="number+gauge",
    value=value,
    number={"suffix": "%"},
    gauge={
        "axis": {"range": [0, 100]},
        "bar": {"color": "red"},
        "steps": [
            {"range": [0, 50], "color": "green"},
            {"range": [50, 100], "color": "red"}
        ]
    }
)

# Set layout properties
layout = go.Layout(
    showlegend=False,
    width=1000,  # Adjust the width of the indicator
    height=333  # Adjust the height of the indicator
)

# Create the figure
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()

# 2022

In [66]:
teams22 = ['seattle', 'seattle', 'seattle', 'seattle', 'usc', 'washington', 'washington', 'washington', 'washington', 'grand-canyon',
        'santa-clara', 'santa-clara', 'santa-clara', 'san-diego', 'uc-santa-barbara', 'uc-santa-barbara', 'uconn', 'uconn',
        'uconn', 'san-diego-state', 'cal-poly', 'cal-poly', 'cal-poly', 'san-diego-state', 'long-beach-state', 'long-beach-state',
        'long-beach-state', 'san-diego', 'uc-irvine', 'uc-irvine', 'uc-irvine', 'uc-santa-barbara', 'uc-santa-barbara', 
        'uc-santa-barbara', 'san-diego-state', 'csu-bakersfield', 'csu-bakersfield', 'csu-bakersfield', 'uc-riverside', 
        'uc-riverside', 'uc-riverside', 'san-diego-state', 'cal-state-fullerton', 'cal-state-fullerton', 'cal-state-fullerton',
        'oregon', 'oregon', 'uc-davis', 'uc-davis', 'uc-davis', 'hawai-i', 'hawai-i', 'hawai-i', 'csun', 'csun', 'csun']
nums22 = [8644, 8645, 8877, 8646, 8647, 8648, 8649, 8897] + list(range(8650, 8656)) + [8916, 8917, 8659, 8660, 8661, 8666, 8663, 8664, 8665, 8662] + list(range(8667, 8699))
triton_innings22 = []
for i in range(56):
    play_by_play_link = 'https://ucsdtritons.com/sports/baseball/stats/2022/' + teams22[i] + '/boxscore/' + str(nums22[i])
    play_soup = bs4.BeautifulSoup(re.get(play_by_play_link).content)
    play_by_play_html = play_soup.find('section', attrs={'id': 'play-by-play'})
    innings = play_by_play_html.find('div', attrs={'id': 'inning-all'}).find_all('div', attrs={'class': 'panel relative'})
    triton_innings22 += (list(filter(lambda x: x.find('caption').text.find('UC San Diego') > -1, innings)))
triton_innings22

[<div class="panel relative">
 <div class="row pad">
 <table class="sidearm-table play-by-play">
 <caption>UC San Diego - Bottom of 1st</caption>
 <thead>
 <tr>
 <th class="text-hide" scope="col">Play Description</th>
 <th class="text-center" scope="col">SU</th>
 <th class="text-center" scope="col">UCS</th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td>Fuhrman, M. struck out looking (1-2).</td>
 <td class="text-center">2</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Robinson, C. walked (3-2).</td>
 <td class="text-center">2</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Smith, J. singled to center field (1-0); Robinson, C. advanced to second.</td>
 <td class="text-center">2</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Lucchetti, A hit by pitch (0-1); Smith, J. advanced to second; Robinson, C. advanced to third.</td>
 <td class="text-center">2</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Filia, M. flied out to cf, SAC, RBI (2-2); Robinson, C. scored.</td>
 <t

In [68]:
names22 = ['Gonzalez', 'Smith', 'Filia', 'McNally', 'Halbach', 'Fuhrman', 'Robinson', 'FitzGerald', 'Saunders', 'Glascoe', 'Sudyka',
        'Kleszcz', 'Lucchetti', 'Costello', 'Solis', 'Fejeran', 'Dickman', 'Hasegawa', 'Mitchell', 'Rocha']
situations22 = get_situations(triton_innings22, names22)
situations22

,batter,first,second,third,outs,total_runs_inn,runs_up_to_batter
0,Fuhrman,_,_,_,0,1,0
1,Robinson,_,_,_,1,1,0
2,J Smith,Robinson,_,_,1,1,0
3,Lucchetti,J Smith,Robinson,_,1,1,0
4,Filia,Lucchetti,J Smith,Robinson,1,1,0
...,...,...,...,...,...,...,...
2277,Halbach,_,_,_,1,1,1
2278,Saunders,_,_,_,2,1,1
2279,Gonzalez,_,_,_,0,0,0
2280,Costello,_,_,_,1,0,0


In [69]:
situations22['runs_after_batter'] = situations22['total_runs_inn'] - situations22['runs_up_to_batter']
situations22['at_least_1_scored'] = situations22['runs_after_batter'] >= 1
situations_base22 = situations22.copy()
situations_base22['first'] = situations_base22['first'].apply(lambda x: '1' if x != '_' else x)
situations_base22['second'] = situations_base22['second'].apply(lambda x: '2' if x != '_' else x)
situations_base22['third'] = situations_base22['third'].apply(lambda x: '3' if x != '_'else x)
situations_base22['base'] = situations_base22['first'] + ' ' + situations_base22['second'] + ' ' + situations_base22['third']
situations_base22

,batter,first,second,third,outs,total_runs_inn,runs_up_to_batter,runs_after_batter,at_least_1_scored,base
0,Fuhrman,_,_,_,0,1,0,1,True,_ _ _
1,Robinson,_,_,_,1,1,0,1,True,_ _ _
2,J Smith,1,_,_,1,1,0,1,True,1 _ _
3,Lucchetti,1,2,_,1,1,0,1,True,1 2 _
4,Filia,1,2,3,1,1,0,1,True,1 2 3
...,...,...,...,...,...,...,...,...,...,...
2277,Halbach,_,_,_,1,1,1,0,False,_ _ _
2278,Saunders,_,_,_,2,1,1,0,False,_ _ _
2279,Gonzalez,_,_,_,0,0,0,0,False,_ _ _
2280,Costello,_,_,_,1,0,0,0,False,_ _ _


# 2021

In [67]:
teams21 = ['san-diego-state', 'san-diego-state', 'san-diego-state', 'san-diego-state', 'san-diego', 'san-diego', 'san-diego', 
        'san-diego', 'santa-clara', 'santa-clara', 'santa-clara', 'santa-clara', 'cal-poly', 'cal-poly', 'cal-poly', 'cal-poly', 
        'long-beach-state', 'long-beach-state', 'long-beach-state', 'long-beach-state', 'uc-irvine', 'uc-irvine', 'uc-irvine', 
        'uc-irvine', 'uc-santa-barbara', 'uc-santa-barbara', 'uc-santa-barbara', 'uc-santa-barbara', 'csu-bakersfield', 
        'csu-bakersfield', 'csu-bakersfield', 'csu-bakersfield', 'uc-riverside', 'uc-riverside', 'uc-riverside', 'uc-riverside',
        'cal-state-fullerton', 'cal-state-fullerton', 'cal-state-fullerton', 'cal-state-fullerton', 'uc-davis', 'uc-davis', 
        'uc-davis', 'uc-davis', 'hawai-i', 'hawai-i', 'hawai-i', 'hawai-i', 'csun', 'csun', 'csun', 'csun']
nums21 = list(range(8197, 8209)) + list(range(8256, 8262)) + [8263, 8267, 8270, 8273, 8275, 8278] + list(range(8284, 8290)) + [8292, 8294] + list(range(8296, 8308)) + list(range(8323, 8331))
triton_innings21 = []
for i in range(52):
    play_by_play_link = 'https://ucsdtritons.com/sports/baseball/stats/2021/' + teams21[i] + '/boxscore/' + str(nums21[i])
    play_soup = bs4.BeautifulSoup(re.get(play_by_play_link).content)
    play_by_play_html = play_soup.find('section', attrs={'id': 'play-by-play'})
    innings = play_by_play_html.find('div', attrs={'id': 'inning-all'}).find_all('div', attrs={'class': 'panel relative'})
    triton_innings21 += (list(filter(lambda x: x.find('caption').text.find('UC San Diego') > -1, innings)))
triton_innings21

[<div class="panel relative">
 <div class="row pad">
 <table class="sidearm-table play-by-play">
 <caption>UC San Diego - Top of 1st</caption>
 <thead>
 <tr>
 <th class="text-hide" scope="col">Play Description</th>
 <th class="text-center" scope="col">UCS</th>
 <th class="text-center" scope="col">SDS</th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td>Lau struck out looking (2-2).</td>
 <td class="text-center">0</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Baumgartner struck out swinging (2-2).</td>
 <td class="text-center">0</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Lucchetti grounded out to 2b (0-2).</td>
 <td class="text-center">0</td>
 <td class="text-center">0</td>
 </tr>
 </tbody>
 </table>
 <dl class="special-stats emphasize inline text-right">
 <dt>Runs</dt>
 <dd>0</dd>
 <dt>Hits</dt>
 <dd>0</dd>
 <dt>Errors</dt>
 <dd>0</dd>
 <dt>Left On Base</dt>
 <dd>0</dd>
 </dl>
 </div>
 </div>,
 <div class="panel relative">
 <div class="row pad">
 <table class="sidearm-table p

In [70]:
names21 = ['Fuhrman', 'Lucchetti', 'Kritsch', 'Baumgartner', 'Soderstrom', 'Kleszcz', 'Gozzo', 'Bynam', 'Stewart', 'Kim', 
          'White', 'Glascoe', 'Filia', 'Lau', 'Mcnally', 'Weiner', 'Sudyka', 'Fitzgerald', 'Hawks', "O'Brien", 'Dickman', 'Velasquez']
situations21 = get_situations(triton_innings21, names21)
situations21['runs_after_batter'] = situations21['total_runs_inn'] - situations21['runs_up_to_batter']
situations21['at_least_1_scored'] = situations21['runs_after_batter'] >= 1
situations_base21 = situations21.copy()
situations_base21['first'] = situations_base21['first'].apply(lambda x: '1' if x != '_' else x)
situations_base21['second'] = situations_base21['second'].apply(lambda x: '2' if x != '_' else x)
situations_base21['third'] = situations_base21['third'].apply(lambda x: '3' if x != '_'else x)
situations_base21['base'] = situations_base21['first'] + ' ' + situations_base21['second'] + ' ' + situations_base21['third']
situations_base21

,batter,first,second,third,outs,total_runs_inn,runs_up_to_batter,runs_after_batter,at_least_1_scored,base
0,Lau,_,_,_,0,0,0,0,False,_ _ _
1,Baumgartner,_,_,_,1,0,0,0,False,_ _ _
2,Lucchetti,_,_,_,2,0,0,0,False,_ _ _
3,Gozzo,_,_,_,0,0,0,0,False,_ _ _
4,Soderstrom,_,_,_,1,0,0,0,False,_ _ _
...,...,...,...,...,...,...,...,...,...,...
2094,Baumgartner,_,_,_,2,0,0,0,False,_ _ _
2095,Soderstrom,_,_,_,0,4,0,4,True,_ _ _
2096,Kim,1,_,_,0,4,0,4,True,1 _ _
2097,Kritsch,1,2,3,0,4,0,4,True,1 2 3


In [74]:
sits2123 = pd.concat([situations_base, situations_base22, situations_base21])
sits2123.to_csv(path_or_buf = 'baseball_atbats_2021-2023.csv', index = False)
sits2123

,batter,first,second,third,outs,total_runs_inn,runs_up_to_batter,runs_after_batter,at_least_1_scored,base
0,Saunders,_,_,_,0,0,0,0,False,_ _ _
1,Smith,_,_,_,1,0,0,0,False,_ _ _
2,Halbach,_,_,_,2,0,0,0,False,_ _ _
3,Kane,_,_,_,0,0,0,0,False,_ _ _
4,Fuhrman,1,_,_,0,0,0,0,False,1 _ _
...,...,...,...,...,...,...,...,...,...,...
2094,Baumgartner,_,_,_,2,0,0,0,False,_ _ _
2095,Soderstrom,_,_,_,0,4,0,4,True,_ _ _
2096,Kim,1,_,_,0,4,0,4,True,1 _ _
2097,Kritsch,1,2,3,0,4,0,4,True,1 2 3


In [75]:
re_matrix_all = pd.pivot_table(data = sits2123, values = ['runs_after_batter'], index = ['base'], columns = ['outs'], aggfunc = 'mean')
re_matrix_all

runs_after_batter                    
outs                  0         1         2
base                                       
1 2 3          2.955556  1.607143  0.780488
1 2 _          1.543307  0.944444  0.512821
1 _ 3          2.050000  1.360000  0.463158
1 _ _          1.043796  0.702820  0.326877
_ 2 3          2.131579  1.720430  0.783505
_ 2 _          1.258065  0.724138  0.343511
_ _ 3          1.750000  1.163265  0.513274
_ _ _          0.676593  0.409715  0.147103

In [76]:
rp_matrix_all = pd.pivot_table(data = sits2123, values = ['at_least_1_scored'], index = ['base'], columns = ['outs'], aggfunc = 'mean')
rp_matrix_all

at_least_1_scored                    
outs                  0         1         2
base                                       
1 2 3          0.888889  0.642857  0.325203
1 2 _          0.677165  0.393519  0.241758
1 _ 3          0.825000  0.670000  0.242105
1 _ _          0.472019  0.323210  0.154964
_ 2 3          0.815789  0.720430  0.340206
_ 2 _          0.612903  0.399015  0.232824
_ _ 3          0.892857  0.714286  0.327434
_ _ _          0.324100  0.201690  0.081724